# Impacto de la criminalidad en el mercado inmobiliario en Jalisco

Se recopilaron bases de datos de viviendas del estado de Jalisco de sus 5 principales municipios:
* Guadalajara
* Zapopan
* Tlajomulco de Zúñiga
* Tónala
* Tlaquepaque

Obtenidas del portal de [vivanuncios.com.mx](https://www.vivanuncios.com.mx/) mediante mineria de datos.\
Donde obtuvimos las variables estructurales del inmueble (*Z) y del entorno (A*)\
Posteriormente se obtuvieron los indices de criminalidad de tres diferentes tipos de robo:
* Robo casa habitación
* robo a comercio
* robo de vehículo automotor.

Obtenidas del portal: [onc.org.mx](https://onc.org.mx)

In [ ]:
import pandas as pd

municipios = ["tlaquepaque", "tonala", "tlajomulco", "guadalajara", "zapopan"]
propiedades_jalisco = pd.read_csv("Data/jalisco/propiedades_jalisco.csv")

dfs = []
# Unimos los datos de las variables de entorno de cada municipio
for municipio in municipios:
    df_municipio = pd.read_csv(f"Data/jalisco/direcciones_lineales/direcciones_lineales_{municipio}.csv")
    dfs.append(df_municipio)
df_entorno = pd.concat(dfs)
df_entorno

In [ ]:
df_estructurales = pd.read_csv("Data/jalisco/propiedades_jalisco.csv")

df = df_estructurales.merge(df_entorno, on="direccion")

# Eliminamos las filas con valores nulos
df = df.dropna()
df.describe()

In [ ]:
import matplotlib.pyplot as plt

df["municipio"].value_counts().plot.pie(autopct='%1.1f%%', figsize=(6, 6), title='Distribución')
plt.plot()

In [ ]:
df["jardin"].value_counts().plot.pie(autopct='%1.1f%%', figsize=(6, 6), title='Distribución')
plt.plot()

In [ ]:
import plotly.express as px

px.box(df, y=['precio'], title='Distribución de precio')


In [ ]:
from scipy.stats import zscore

# Eliminamos la propiedad de $125 MXN
df = df[df['precio'] != 125]
df["precio"].describe()

In [ ]:
df[df["precio"] <= 400000]

In [ ]:
df = df[df['precio'] != 4950]
df["precio"].describe()

In [ ]:
import seaborn as sns

sns.violinplot(df["precio"], inner="box")
plt.show()

Repetimos el ejercicio para la variable `m2_lote` que también presenta valores atípicos.

In [ ]:
px.box(df, y=['m2_lote'], title='Distribución de m2_lote')

In [ ]:
df = df[df['m2_lote'] != 1110000]
zscores_lote = zscore(df['m2_lote'])

df = df[abs(zscores_lote) < 3]
df["m2_lote"].describe()

In [ ]:
sns.violinplot(df["m2_lote"], inner="box")

Finalmente, repetimos el procedimiento para la variable `Distancia la centro de la ciudad (km)` que presenta un valor atípico.

In [ ]:
px.box(df, y=['Distancia al centro de la ciudad (km)'], title='Distribución de distancia al centro de la ciudad (km)')

In [ ]:
df = df[df['Distancia al centro de la ciudad (km)'] != 7782.9]
zscores_centro = zscore(df['Distancia al centro de la ciudad (km)'])

df = df[abs(zscores_centro) < 3]
df["Distancia al centro de la ciudad (km)"].describe()

In [ ]:
sns.violinplot(df["Distancia al centro de la ciudad (km)"], inner="box")

In [ ]:
df.describe()

In [ ]:
df["municipio"].value_counts().plot.pie(autopct='%1.1f%%', figsize=(6, 6), title='Distribución')
plt.plot()

In [ ]:
# Agregamos los indices de criminalidad

indices_criminalidad = {
    'guadalajara': {'robo_de_vehiculo_automotor':192.69, 'robo_a_negocio':80.67, 'robo_casa_habitacion':21.82},
    'tlaquepaque': {'robo_de_vehiculo_automotor':220.06, 'robo_a_negocio':67.25, 'robo_casa_habitacion':45.35},
    'tonala': {'robo_de_vehiculo_automotor':108.46, 'robo_a_negocio':33.36, 'robo_casa_habitacion':16.76},
    'tlajomulco': {'robo_de_vehiculo_automotor':91.67, 'robo_a_negocio':67.88, 'robo_casa_habitacion':58.60},
    'zapopan': {'robo_de_vehiculo_automotor':97.58, 'robo_a_negocio':44.57, 'robo_casa_habitacion':16.27}
}

# Convertir a DataFrame y poner municipio como columna
df_criminalidad = pd.DataFrame.from_dict(indices_criminalidad, orient='index').reset_index()
df_criminalidad = df_criminalidad.rename(columns={'index':'municipio'})

df = df.merge(df_criminalidad, on='municipio', how='left')
df

In [ ]:
df['jardin'] = df['jardin'].astype(int)
df.to_csv("Data/clean_data.csv", index=False, encoding="utf-8")
df.describe()

In [ ]:
means = []
pd.set_option('display.float_format', '{:.2f}'.format)

for municipio in municipios:
    means.append(df[df['municipio']==municipio].select_dtypes(include=['number']).mean())

df_means = pd.DataFrame(means)
df_means.select_dtypes(include=['number']).mean()
